In [2]:
import cffi
import os
import numpy as np
import pandas as pd
from io import StringIO
from enum import Enum
from Bio import SeqIO
from Bio import Entrez
from pathlib import Path
import matplotlib.pyplot as plt
from multiprocessing import Pool
import glob
import re
import fnmatch
from src.SequenceProcessor import SequenceProcessor
from src.FileHandler import FileHandler
from src.pytrsomix import SeqAnalyzer,TRScalculator
from src.BlastProcessor import BLASTProcessor
import argparse
import time

In [ ]:
def map_accession_to_taxid(accession):
    """Map accession to TaxID using the optimized lookup dictionary."""
    return accession_to_taxid.get(accession, '') 
 
def main():
    start_time = time.time()
    parser = argparse.ArgumentParser(description='''This program analyzes the blast output files to find species specific sequences coming from genomes
                                     analyzed in the previous step''')
    parser.add_argument('--blast_output_path', help="Path to a folder containing blast results in a valid format", type=str, required=True)
    parser.add_argument('--taxonomy_db',help='Path to the taxonomy - accession database', required=True, type= str)
    args = parser.parse_args()

    blast_output = args.blast_output_path
    taxonomy_db = args.taxonomy_db

    FileHandler.convert_to_txt(blast_output)
    FileHandler.filter_and_overwrite_blast_file(blast_output)
    blast_output_path = Path(blast_output)
    results_directory = blast_output_path.parent


    '''
    Get accessions present in blast files and filter the taxonomy_file using them this way we have all accession - taxid
    Pairs in out dataset and there is not need to access such a large database.
    In addition as you can see I'm using chunks to process the initial database, that size could be altered but i found 
    50000 to be sweet spot. Loading progress bar was added for this operation as well.
    '''
    taxonomy_file = taxonomy_db
    accessions = BLASTProcessor.collect_accessions_from_blast_files(blast_output)
    tax_df = SequenceProcessor.filter_taxonomy_file(taxonomy_file,accessions,50000)
    
    '''
    Saving the filtered database to a csv file for use later - NOT IMPLEMENTED
    '''
    
    taxonomy_dataframe = os.path.join(blast_output_path, "taxonomy_filtered.csv")
    tax_df.to_csv(taxonomy_dataframe, index=False)
    print(f"tax_df saved to {taxonomy_dataframe}")

    '''
    Quite self-explanatory makes sure that each taxid - accession pair gets put into dictionary 
    only once i make use of sets here to decrease it's size and repeated information
    '''
    taxid_accessions_dict = {}
    for index, row in tax_df.iterrows():
        accession_column = tax_df.columns[0]  # Extract accession column dynamically
        taxid_column = tax_df.columns[1]  # Extract taxid column dynamically
        
        accession = row[accession_column]
        taxid = row[taxid_column]
        
        if taxid in taxid_accessions_dict:
            taxid_accessions_dict[taxid].append(accession)
        else:
            taxid_accessions_dict[taxid] = [accession]
    # Ensure the directory for modified files exists
    modified_blast_path = os.path.join(blast_output_path, "modified_blast")
    FileHandler.ensure_directory_exists(modified_blast_path)  # Ensure this function creates the directory if it doesn't exist

    '''
    This step assumes each accession maps to exactly one taxid which is true (but that matching is not always accurate)
    this drawback is attributed to nucleotide database structure and is not something that should alter the results in any
    significant way as long as we remember to add the top-level species to the dictionary later
    '''

    accession_to_taxid = {accession: taxid for taxid, accession in taxid_accessions_dict.items() for accession in accessions}

    # Iterate over files in the input directory
    for filename in os.listdir(blast_output_path):
        if filename.endswith(".txt"):
            input_file_path = os.path.join(blast_output_path, filename)
            
            # Read the file into a DataFrame
            df = pd.read_csv(input_file_path, sep='\t', header=None)  # No column names specified
            
            # Map accessions to TaxIDs and add them as a new column after the last existing column
            df[df.shape[1]] = df[1].map(map_accession_to_taxid)  # Assuming column 1 contains the accessions
            
            # Construct output file path
            modified_file_path = os.path.join(modified_blast_path, f"taxids_{filename}")
            
            # Save the modified DataFrame to a new file in the output directory
            df.to_csv(modified_file_path, sep='\t', index=False, header=False)
    '''This is still experimental but should allow to load interiors.txt equivalent into a dataframe to construct the
    dictionary of genomes which we analyze.'''
    print("Searching for the results file....")
    results_file = FileHandler.find_file_by_name('*_results.csv',folder = results_directory, auto=True)
    if results_file:
        print(f"Selected file: {results_file}")
    else:
        # If results_file is either None or an empty string, prompt the user for the path.
        print("No '*_results.csv' file selected or found.")
        results_file = input("Please provide the path to the current experiments result file: ")

    combined_results = pd.read_csv(results_file)
